# Bell-LaPadula models for confidentiality of public health data

*J. True Merrill, Georgia Tech Research Insitute*

This document outlines the application of the [Bell-LaPadula](https://en.wikipedia.org/wiki/Bell%E2%80%93LaPadula_model) model as a formal system for safeguarding the confidentiality of public health data stored in information systems. The model, rooted in state machines, serves as a mathematical framework to articulate a manditory access control (MAC) policy.  Originally developed as a formal method to study the security of US Department of Defense computer systems, the Bell-LaPadula model serves as the theoretical underpinning of nearly all MAC systems, including [Security Enhanced Linux](https://en.wikipedia.org/wiki/Security-Enhanced_Linux) (SELinux).

## Formalism

We begin by describing notation and concepts. 

The information system involves multiple users working concurrently on shared data. To articulate the Mandatory Access Control (MAC) policy, each user and data component receives a set of **classification categories**.  A classification category is a label or tag that is used as an annotaion to enforce access control rules.

Importantly, we do not assume the classification categories form an ordinal set, where classifications can be ordered in an increasing hiearchy of protection levels.  This differs from the orignal Bell-LaPaulda model, which used the US Department of Defense (DoD) classification system (`UNCLASSIFIED` < `CONFIDENTIAL` < `SECRET` < `TOP SECRET`).  In the DoD system, because the classification categories are ordered, it possible to talk of data having "higher" or "lower" classification level.  This is not always possible with a non-ordered system.

### Subjects and Objects

We introduce definitions rooted in Bell and LaPaulda's original approach.  A subject is defined by the following properties:

* A subject $S_i$ is a computer process launched by a user or by another subject.
* A subject's classification is the set $C(S_i) \subseteq \Omega$, where $\Omega$ is the set of all classification categories.
* Some subjects are trusted. A trusted subject is annotated with a star as $S_i^*$.

An object is defined by the following properties:

* An object $O_j$ is data such as a file or a field in a database.
* An object's classification is the set $C(O_j) \subseteq \Omega$.

When we say that a classification set $C_i$ is **lower** than another set $C_j$, we mean that $C_i \subset C_j$.  Similarly if $C_i$ has a **higher** classification than $C_j$ then $C_i \supset C_j$.  Classifications are equal if and only if $C_i = C_j$.  But notably if the two classifications are not subsets of each other, we cannot say that one is higher than the other, only that they are distinct classifications.

### Security properties

The MAC system enforces the following security properties:

* **The simple property (no read up)**: A subject $S_i$ may read an object $O_j$ if and only if $C(S_i) \supseteq C(O_j)$. Simply put, a subject can only read objects at the same or lower classification level.  This property is required so that subjects do not access information they are not authorized to.

* **The $*$-property (no write down)**: An untrusted subject $S_i$ may write an object $O_j$ if and only if $C(S_i) \subseteq C(O_j)$. Simply put, an untrusted subject can only write objects at the same or higher classification level.  This property is required so that subjects do not leak protected information to unprivliged channels.

* **The $*$-property (for trusted subjects)**: A trusted subject $S_i^*$ may write an object $O_j$ if and only if it has read access to the object. Trusted subjects are subjects that the system administrator has a high degree of confidence will not leak protected information to an unprivliged channel.  In practice, users are often trusted subjects while computer procceses launched by users are often not trusted.  Trusted subjects are needed in most useful MAC systems, as without them the access controls are too limiting to accomplish practical work.

### An example with public health data

To help make things less abstract, we will refer to an example case of a MAC policy for public health data.  In this example, we will assume we are working with a system that stores Personal Health Information (PHI), HIPAA data, and data protected by two separate data sharing agreements.  The set of all classification categories is

$$
  \Omega = \{ \texttt{PHI},\: \texttt{HIPAA},\: \texttt{DSA1},\: \texttt{DSA2} \}
$$

where $\texttt{DSA1}$ and $\texttt{DSA2}$ are classification categories for the first and second data sharing agreements respectively.  Now suppose we are working documents with the classifications

| Document | Classification |
|----------|------------------------|
| $O_1$    | $\{ \texttt{PHI} \}$  |
| $O_2$    | $\{ \texttt{PHI},\: \texttt{DSA1} \}$ |
| $O_3$    | $\{ \texttt{HIPAA},\: \texttt{DSA2} \}$ |

From these definitions we can infer the following:

* Document $O_1$ is at a lower classification than $O_2$ since $\{ \texttt{PHI} \} \subset \{ \texttt{PHI},\: \texttt{DSA1} \}$.
* Documents $O_2$ and $O_3$ are at different classifications since $C(O_2) \neq C(O_3)$, but neither classiciation is higher than the other because $C(O_2) \not\subset C(O_3)$ and $C(O_2) \not\supset C(O_3)$.

Now let $S_1$ and $S_2^*$ denote two users with the following access levels

| User | Classification |
|------|----------------|
| $S_1$ | $\{ \texttt{PHI},\: \texttt{DSA1} \}$ |
| $S_2^*$ | $\{ \texttt{PHI},\: \texttt{HIPAA},\: \texttt{DSA2} \}$ |

The user $S_2^*$ is considered trusted.  Given these levels, the users access to the documents is given by the following truth table.

| User  | Document | Read Access | Write Access |
|-------|----------|-------------|--------------|
| $S_1$ | $O_1$    | True        | False        |
| $S_1$ | $O_2$    | True        | True         |
| $S_1$ | $O_3$    | False       | False        |
| $S_2$ | $O_1$    | True        | True         |
| $S_2$ | $O_2$    | False       | False        |
| $S_2$ | $O_3$    | True        | True         |


## Implementation in Python

Below, we build a simple MAC system in the Python programming language.

In [1]:
from typing import Any, Generic, Iterable, Protocol, Set, TypeVar


Classification = Set[str]
"""Type alias for a classification label"""


class SupportsClassification(Protocol):
    """A protocol for objects that support Classification"""

    _classification: Classification


T = TypeVar("T")
V = TypeVar("V", bound=SupportsClassification)


class ClassifiedObject(Generic[T]):
    """A type for objects"""

    def __init__(self, value: T, classification: Classification):
        """Initialize an object
        
        Args:
            value (T): the object's value
            classification (Classification): the object's classification
        """
        self._classification = classification
        self.value = value


class Subject:
    """A type for subjects"""

    def __init__(self, classification: Classification, trusted: bool = False):
        """Initialize a subject

        Args:
            classification (Classification): the subject's classification
                access
            trusted (bool, optional): whether the subject is trusted.  Defaults
                to False.
        """
        self._classification = classification
        self._trusted = trusted

    @property
    def trusted(self):
        return self._trusted


def get_classification(obj: SupportsClassification) -> Classification:
    """Get the classification of an object

    Args:
        obj (SupportsClassification): the object to get the classification of

    Returns:
        Classification: the classification of the object
    """
    return getattr(obj, "_classification", set())


def set_classification(obj: V, classification: Classification) -> V:
    """Set the classification of an object

    Args:
        obj (V): the object to classify
        classification (Classification): the classification

    Returns:
        V: the object with the classification applied.
    """
    setattr(obj, "_classification", classification)
    return obj


def can_read(sub: Subject, obj: SupportsClassification) -> bool:
    """Check whether `sub` can read `obj`

    Args:
        sub (Subject): the subject
        obj (SupportsClassification): the object

    Returns:
        bool: whether the subject can read the object
    """
    cs = get_classification(sub)
    co = get_classification(obj)
    return (cs >= co)


def can_write(sub: Subject, obj: SupportsClassification) -> bool:
    """Check whether `sub` can write to `obj`

    Args:
        sub (Subject): the subject
        obj (SupportsClassification): the object

    Returns:
        bool: whether the subject may write to the object
    """
    if sub.trusted:
        return can_read(sub, obj)
    
    cs = get_classification(sub)
    co = get_classification(obj)
    return (cs <= co)


## Example data

Below, we use the Python implementation to reproduce the truth table we derived in the example earlier

In [2]:
import pandas as pd
import itertools

objects = (
    ClassifiedObject("first_document", {"PHI"}),
    ClassifiedObject("second_document", {"PHI", "DSA1"}),
    ClassifiedObject("third_document", {"HIPAA", "DSA2"})
)

subjects = (
    Subject({"PHI", "DSA1"}),
    Subject({"PHI", "HIPAA", "DSA2"}, trusted=True)
)


def iter_truth_table_rows(subjects, objects):
    for i, sub in enumerate(subjects):
        sub_lab = f"$S_{i + 1}$"
        for j, obj in enumerate(objects):
            obj_lab = f"$O_{j + 1}$"
            yield {
                "user": sub_lab,
                "document": obj_lab,
                "read": can_read(sub, obj),
                "write": can_write(sub, obj)
            }

truth_table = pd.DataFrame(list(iter_truth_table_rows(subjects, objects)))
truth_table

,user,document,read,write
0,$S_1$,$O_1$,True,False
1,$S_1$,$O_2$,True,True
2,$S_1$,$O_3$,False,False
3,$S_2$,$O_1$,True,True
4,$S_2$,$O_2$,False,False
5,$S_2$,$O_3$,True,True


## Constraints on subjects

A subject refers to a computer process initiated by a user or another subject. For instance, a subject might be a login shell process launched by a user during their login to an information system. Subsequent programs, known as child processes, launched from that shell also qualify as subjects.  

Here we explore how the security properties constrain the allowed behavior of a subject and it's children.

### Subjects are functions

A process can abstractly be thought as a function $f: X \rightarrow Y$ which maps data from an input domain $X$ to an output range $Y$.  A process may take multiple input arguments or return multiple outputs.  This means the domain (range) is generally a Cartesian product of sub-domains (sub-ranges) of the form

$$
X = X_1 \times X_2 \times \cdots \times X_n,
$$

$$
Y = Y_1 \times Y_2 \times \cdots \times Y_m.
$$

Subsequently, the classification of the domain (range) is the union of classification of the sub-domain (sub-ranges).

$$
    C(X) = \bigcup_{i = 1}^n C(X_i) \qquad \text{and} \qquad
    C(Y) = \bigcup_{i = 1}^m C(Y_i).
$$

#### Untrusted processes

In order to run, the process must be able to read the input data and write the output data to disk.  The security properties impose the following constraints for untrusted processes,

$$
    C(f) \supseteq C(X) \qquad \text{and} \qquad
    C(f) \subseteq C(Y).
$$

The only way both conditions can be simultaneously true is if $C(X) = C(Y)$.  Hence, untrusted processes are unable to modify the classification of data; their capability is limited to transforming data from one classification to new data of the same classification.

#### Trusted processes

The situation is different for a trusted process $f^*$.  Here the security properties impose $C(f^*) \supseteq C(X)$ and $C(f^*) \supseteq C(Y)$.  This implies $C(f^*) \supseteq C(X) \cup C(Y)$.  Simply put, this requires that the trusted process must have a higher classification than the classifications of the union of the domain and range classifications.

In [3]:
# from abc import ABC, abstractmethod

# class Process(ABC, Subject):
    
#     @abstractmethod
#     def call(self, args: Iterable[ClassifiedObject]) -> ClassifiedObject:
#         """Call the process

#         Args:
#             args (Iterable[ClassifiedObject]): the arcuments